In [1]:
%%capture
pip install ipyleaflet

In [2]:
from ipyleaflet import Map, Marker, Popup, LayerGroup, MarkerCluster, WidgetControl, AwesomeIcon
from ipywidgets import HTML, IntSlider, jslink

In [3]:
import duckdb
duckdb = duckdb.connect('/duckdb/dev.duckdb', config={})
duckdb.query("INSTALL SPATIAL; LOAD SPATIAL")

In [4]:
center = (38, -90)
m = Map(center=center, zoom=4, height='800px')

map_markers = LayerGroup()
m.add(map_markers)

zoom_slider = IntSlider(description='Zoom:', min=4, max=15, value=4)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add(widget_control1)

display(m)

Map(center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [8]:
map_markers.clear_layers()

rows = duckdb.query("select code, station_lat, station_lon, bikeshare_lat, bikeshare_lon, station_name, bikeshare_name from int_amtrak_stations_to_bikeshare where dist < 400 order by dist asc").fetchall()

stations = {}

for row in rows:
    if row[0] not in stations:
        stations[row[0]] = []
    stations[row[0]].append(row[1:])

for code, rows in stations.items():
    markers = []

    message = HTML()
    message.value = f"<b>{code}</b><br>{rows[0][4]}"
    markers.append(Marker(location=(rows[0][0], rows[0][1]), draggable=False, popup=message, icon=AwesomeIcon(name='train')))
    for row in rows:
        message = HTML()
        message.value = f"{row[5]}"
        markers.append(Marker(location=(row[2], row[3]), draggable=False, popup=message, icon=AwesomeIcon(name='bicycle')))

    map_markers.add(MarkerCluster(markers=markers))